In [1]:
import requests
import pandas as pd
import functions
from importlib import reload
reload(functions)

<module 'functions' from '/mnt/c/DEV/Masterarbeit/code/functions.py'>

In [2]:
package_name = "matplotlib"
response = requests.get(f"https://pypi.org/pypi/{package_name}/json")
pypi_data = response.json()

In [3]:
owner, repo, repo_link = functions.get_pypi_repo(pypi_data)
repo_link

'https://github.com/matplotlib/matplotlib'

In [4]:
pd.set_option('display.max_rows', None)

## Git quick stats

In [5]:
git_contributors_df = await functions.get_git_contributors(owner, repo, repo_link, package_name)
git_contributors_df.head(100)

,name,email,insertions,deletions,lines_changed,files,commits,first_commit,last_commit
0,Antony Lee,anntzer.lee@gmail.com,114059,209576,323635,12443,3842,2013-01-04 13:05:35+01:00,2024-08-21 09:59:53+02:00
1,Thomas A CaswellDr. Thomas A Caswell,tcaswell@gmail.com,106781,92853,199634,7317,3539,2012-08-08 15:08:15-05:00,2024-08-16 16:38:12-04:00
2,Michael Droettboom,mdboom@gmail.com,208736,190271,399007,7893,3021,2007-07-03 13:02:53+00:00,2018-11-26 10:32:24-05:00
3,Elliott Sales de Andrade,quantum.analyst@gmail.com,106530,78785,185315,6804,2615,2014-07-06 16:50:50-04:00,2024-08-22 21:29:39-04:00
4,Tim Hoffmann,2836374+timhoffm@users.noreply.github.com,57321,46093,103414,5416,1600,2017-11-13 01:01:44+01:00,2024-08-15 16:45:02+02:00
5,John Hunter,jdh2358@gmail.com,356071,224386,580457,5956,1475,2005-04-11 15:17:26+00:00,2012-06-30 14:02:02-05:00
6,Eric Firing,efiring@hawaii.edu,56323,87645,143968,2842,1297,2005-09-25 23:03:37+00:00,2023-06-05 15:00:48-10:00
7,Jody Klymak,jklymak@gmail.com,49344,25237,74581,3532,1081,2015-06-06 20:45:54-07:00,2024-07-17 07:15:46-07:00
8,David Stansby,dstansby@gmail.com,23832,23061,46893,2812,1039,2016-05-05 12:12:33+01:00,2024-04-07 15:16:58+02:00
9,Jens Hedegaard Nielsen,jenshnielsen@gmail.com,8627,8531,17158,856,575,2011-06-24 14:42:31+02:00,2018-04-23 08:33:51+02:00


## PyPi Owner (verified)

PyPi Owner sind in PyPi immer Organisationen, deswegen brauche ich diese nicht zu verarbeiten.

## PyPi Maintainers (verified)

In [6]:
pypi_maintainers_df = await functions.get_pypi_maintainers(package_name)
pypi_maintainers_df

,login,name
0,ivanov,None
1,matthew.brett,Matthew Brett
2,mdboom2,None


Kann keine logins matchen, da ich die aus git nicht bekomme. Aus dem Grund matche ich jetzt logins mit emails.

In [7]:
result = functions.matching(pypi_maintainers_df, git_contributors_df)
result

,login,name,rank,insertions,deletions,lines_changed,files,commits,first_commit,last_commit,score
2,mdboom2,None,3,208736,190271,399007,7893,3021,2007-07-03 13:02:53+00:00,2018-11-26 10:32:24-05:00,0.5
0,ivanov,None,36,5185,2869,8054,375,96,2011-01-12 00:49:36+00:00,2019-11-06 08:22:12-08:00,0.5
1,matthew.brett,Matthew Brett,53,3191,2264,5455,536,56,2013-08-21 19:51:54-07:00,2017-10-24 13:05:13+01:00,1.0


## Python Authors (not verified)

Assuming Author seperation by ,

In [8]:
python_authors_df = functions.get_python_authors(pypi_data)
python_authors_df

,name,email
0,John D. Hunter,Unknown <matplotlib-users@python.org>
1,Michael Droettboom,None


In [9]:
result = functions.matching(python_authors_df, git_contributors_df)
result

,name,email,rank,insertions,deletions,lines_changed,files,commits,first_commit,last_commit,score
1,Michael Droettboom,None,3,208736,190271,399007,7893,3021,2007-07-03 13:02:53+00:00,2018-11-26 10:32:24-05:00,0.5
0,John D. Hunter,Unknown <matplotlib-users@python.org>,6,356071,224386,580457,5956,1475,2005-04-11 15:17:26+00:00,2012-06-30 14:02:02-05:00,0.5


## Python Maintainers (not verified)

In [10]:
python_maintainers_df = functions.get_python_maintainers(pypi_data)
python_maintainers_df

""


In [11]:
result = functions.matching(python_maintainers_df, git_contributors_df)
result

,rank,insertions,deletions,lines_changed,files,commits,first_commit,last_commit,score


## Citation File Format (Authors)

In [12]:
cff_df = functions.get_cff_authors(owner, repo)
cff_df

,name,email,ORCID,type,date-released
0,The Matplotlib Development Team,None,None,software,None


In [13]:
result = functions.matching(cff_df, git_contributors_df)
result

,name,email,ORCID,type,date-released,rank,insertions,deletions,lines_changed,files,commits,first_commit,last_commit,score
0,The Matplotlib Development Team,None,None,software,None,None,None,None,None,None,None,None,None,0


## Citation File Format (Preferred citation Authors)

In [14]:
cff_df = functions.get_cff_preferred_citation_authors(owner, repo)
cff_df

,name,email,ORCID,type,year,date-published
0,John D. Hunter,None,None,article,2007,2007-06-18


In [15]:
result = functions.matching(cff_df, git_contributors_df)
result

,name,email,ORCID,type,year,date-published,rank,insertions,deletions,lines_changed,files,commits,first_commit,last_commit,score
0,John D. Hunter,None,None,article,2007,2007-06-18,6,356071,224386,580457,5956,1475,2005-04-11 15:17:26+00:00,2012-06-30 14:02:02-05:00,0.5


## Citation.bib

In [16]:
bib_df = functions.get_bib_authors(owner, repo)
bib_df

,name
0,J. D. Hunter


In [17]:
result = functions.matching(bib_df, git_contributors_df)
result

,name,rank,insertions,deletions,lines_changed,files,commits,first_commit,last_commit,score
0,J. D. Hunter,None,None,None,None,None,None,None,None,0
